In [3]:
import glob
import csv
import json
import re
from urllib.request import urlopen
import csv
import spacy
summary = dict()

nlp = spacy.load('en_core_web_md')

def regex(comment):
    p = re.sub(r'[^A-Za-z0-9]+',' ',comment, flags=re.IGNORECASE)
    p = re.sub(r'\s+',' ',p)
    return p

def readfile():
    summary_id = 0
    for filename in glob.iglob('dataset/raw_dataset/*.csv', recursive=True):
        with open(filename) as csvfile:
            reader = csv.DictReader(csvfile)
            print(filename,reader.fieldnames)
            for row in reader:
                summary[summary_id] = dict()
                summary[summary_id]['project'] = filename.split('/')[2].split('.')[0]
                if 'text' in reader.fieldnames:
                    summary[summary_id]['text'] = row['text']
                elif 'sentence' in reader.fieldnames:
                    summary[summary_id]['text'] = row['sentence']
                summary[summary_id]['oracle'] = row['oracle']
                summary_id += 1

def stop_word_removed(comment):
    non_stop_words = []
    doc = nlp(comment)
    for token in doc:
        if token.is_stop:
            continue
        non_stop_words.append(token.text.lower())
    return ' '.join(non_stop_words)

def writefile():
    with open('dataset/dataset.csv', 'w', newline='') as csvfile:
        fieldnames = ['key','project','regular_text','text','oracle']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for summary_id in summary:
            writer.writerow({
                'key': summary_id, 
                'project': summary[summary_id]['project'].lower(),
                'regular_text': regex(summary[summary_id]['text']),
                'text': regex(stop_word_removed(summary[summary_id]['text'])), 
                'oracle': summary[summary_id]['oracle']
            })
readfile()
writefile()

do not
